In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000


c1 = .005
c2 = .0025


# a  = .25, k= 9

In [5]:
a = 0.25
num_params=9


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.94 & 0.91 & 0.94 & 0.97   \\
Model 1 & 0.06 & 0.06 & 0.05 & 0.09 & 0.06 & 0.03   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.94 , 0.056, 0.004]), array([0.94 , 0.056, 0.004]), array([0.944, 0.05 , 0.006]), array([0.906, 0.092, 0.002]), array([0.936, 0.062, 0.002]), array([0.97 , 0.028, 0.002]), 2.668847952310612, 4.895193672974873, 4.888450298794136)


In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.84 & 0.85 & 0.80 & 0.84 & 0.89   \\
Model 1 & 0.15 & 0.15 & 0.15 & 0.20 & 0.16 & 0.11   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.844, 0.15 , 0.006]), array([0.844, 0.15 , 0.006]), array([0.848, 0.146, 0.006]), array([0.798, 0.196, 0.006]), array([0.838, 0.158, 0.004]), array([0.888, 0.11 , 0.002]), 6.297288586061813, 6.716062445728239, 6.688713568551228)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.98 & 0.98 & 0.96 & 0.98 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.03 & 0.02 & 0.01   \\
Model 2 & 0.02 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.972, 0.012, 0.016]), array([0.976, 0.012, 0.012]), array([0.984, 0.008, 0.008]), array([0.962, 0.034, 0.004]), array([0.976, 0.02 , 0.004]), array([0.994, 0.006, 0.   ]), -0.35178156138517275, 3.3593414469861536, 3.37327469845312)


# a  = .25, k= 4, something not right?

In [9]:
a = 0.25
num_params=4


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.96 & 0.97 & 0.97 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.03 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.956, 0.01 , 0.034]), array([0.956, 0.01 , 0.034]), array([0.958, 0.008, 0.034]), array([0.968, 0.01 , 0.022]), array([0.974, 0.008, 0.018]), array([0.992, 0.004, 0.004]), -1.704711974902755, 5.70811481833208, 5.619872902265575)


# a  = .25, K = 19

In [11]:
a = 0.25
num_params=19


In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.91 & 0.91 & 0.91 & 0.83 & 0.88 & 0.96   \\
Model 1 & 0.09 & 0.09 & 0.09 & 0.17 & 0.12 & 0.04   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.906, 0.094, 0.   ]), array([0.906, 0.094, 0.   ]), array([0.914, 0.086, 0.   ]), array([0.828, 0.172, 0.   ]), array([0.876, 0.124, 0.   ]), array([0.964, 0.036, 0.   ]), 4.22570946455072, 4.398882648828686, 4.525041744553635)


# evidence of power

In [13]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [14]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.54 & 0.54 & 0.59 & 0.44 & 0.51 & 0.71   \\
Model 1 & 0.46 & 0.46 & 0.41 & 0.56 & 0.49 & 0.29   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.544, 0.456, 0.   ]), array([0.544, 0.456, 0.   ]), array([0.592, 0.408, 0.   ]), array([0.436, 0.564, 0.   ]), array([0.512, 0.488, 0.   ]), array([0.712, 0.288, 0.   ]), 9.547848539427063, 4.896609416452536, 4.989962405588145)


In [15]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.54 & 0.54 & 0.56 & 0.38 & 0.52 & 0.70   \\
Model 1 & 0.46 & 0.46 & 0.44 & 0.62 & 0.48 & 0.30   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.544, 0.456, 0.   ]), array([0.544, 0.456, 0.   ]), array([0.562, 0.438, 0.   ]), array([0.384, 0.616, 0.   ]), array([0.518, 0.482, 0.   ]), array([0.704, 0.296, 0.   ]), 9.599051757651146, 4.532031098669451, 4.970141782046844)


In [16]:
a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.49 & 0.50 & 0.58 & 0.46 & 0.51 & 0.69   \\
Model 1 & 0.51 & 0.50 & 0.42 & 0.54 & 0.49 & 0.31   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.488, 0.512, 0.   ]), array([0.498, 0.502, 0.   ]), array([0.584, 0.416, 0.   ]), array([0.458, 0.542, 0.   ]), array([0.512, 0.488, 0.   ]), array([0.688, 0.312, 0.   ]), 9.424664244955329, 4.533898035647223, 4.721620187336325)


In [17]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.54 & 0.54 & 0.59 & 0.40 & 0.51 & 0.72   \\
Model 1 & 0.46 & 0.46 & 0.41 & 0.60 & 0.49 & 0.28   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.542, 0.458, 0.   ]), array([0.542, 0.458, 0.   ]), array([0.586, 0.414, 0.   ]), array([0.398, 0.602, 0.   ]), array([0.51, 0.49, 0.  ]), array([0.718, 0.282, 0.   ]), 9.528814714421115, 5.075931045935975, 4.938979650454393)


In [18]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.56 & 0.56 & 0.58 & 0.42 & 0.52 & 0.71   \\
Model 1 & 0.44 & 0.44 & 0.42 & 0.58 & 0.48 & 0.29   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.556, 0.444, 0.   ]), array([0.556, 0.444, 0.   ]), array([0.576, 0.424, 0.   ]), array([0.416, 0.584, 0.   ]), array([0.524, 0.476, 0.   ]), array([0.706, 0.294, 0.   ]), 9.38603494712957, 5.103394163701947, 4.899812285618196)


# evidence of power 2

In [19]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.69 & 0.69 & 0.71 & 0.86 & 0.92 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.31 & 0.31 & 0.29 & 0.14 & 0.08 & 0.03   \\
\hline
\end{tabular}
(array([0.69, 0.  , 0.31]), array([0.694, 0.   , 0.306]), array([0.714, 0.   , 0.286]), array([0.86, 0.  , 0.14]), array([0.916, 0.   , 0.084]), array([0.972, 0.   , 0.028]), -6.492021472912407, 3.7863281042683026, 3.6423339371233254)


In [20]:
a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.74 & 0.74 & 0.76 & 0.89 & 0.96 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.26 & 0.26 & 0.24 & 0.11 & 0.04 & 0.02   \\
\hline
\end{tabular}
(array([0.742, 0.   , 0.258]), array([0.744, 0.   , 0.256]), array([0.756, 0.   , 0.244]), array([0.888, 0.   , 0.112]), array([0.956, 0.   , 0.044]), array([0.984, 0.   , 0.016]), -6.272482654307053, 3.40963958606361, 3.68068463786367)


In [21]:
a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.66 & 0.73 & 0.73 & 0.87 & 0.91 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.34 & 0.27 & 0.27 & 0.13 & 0.09 & 0.02   \\
\hline
\end{tabular}
(array([0.662, 0.   , 0.338]), array([0.726, 0.   , 0.274]), array([0.732, 0.   , 0.268]), array([0.87, 0.  , 0.13]), array([0.91, 0.  , 0.09]), array([0.976, 0.   , 0.024]), -6.234530009436514, 3.541553594389252, 3.498676292313182)


In [22]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.21 & 0.22 & 0.27 & 0.48 & 0.58 & 0.74   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.79 & 0.78 & 0.73 & 0.52 & 0.42 & 0.26   \\
\hline
\end{tabular}
(array([0.214, 0.   , 0.786]), array([0.222, 0.   , 0.778]), array([0.27, 0.  , 0.73]), array([0.484, 0.   , 0.516]), array([0.576, 0.   , 0.424]), array([0.742, 0.   , 0.258]), -12.086254588941243, 5.010277003688951, 4.858361895079873)


In [23]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.91 & 0.92 & 0.93 & 0.98 & 0.99 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.09 & 0.08 & 0.07 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.914, 0.   , 0.086]), array([0.918, 0.   , 0.082]), array([0.926, 0.   , 0.074]), array([0.978, 0.   , 0.022]), array([0.988, 0.   , 0.012]), array([1., 0., 0.]), -3.8796162331525945, 2.732340673718245, 2.931944406831849)
